# 2022 Sydney University Honours project - Novel kernels for deep learning on light field images

In [2]:
import numpy as np
import os
import PIL
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import sys, glob, os, random
import pandas as pd
import scipy.io as sio
import preprocessing.flatten
import preprocessing.hci_dataset_tools.file_io as hci_io
print('tensorflow version: ', tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


tensorflow version:  2.9.1
Num GPUs Available:  1


2022-07-10 17:33:27.959721: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-10 17:33:27.985668: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-10 17:33:27.985812: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


## Data Loading and Prepreprocessing

In [5]:
data_path = '../../datasets'
hci_boxes = '/hci_dataset/training/boxes/'
hci_boxes_stacked = '/hci_dataset/training/boxes/stacked/'

def load_dataset(num_imgs=1, read_dirs=[data_path+hci_boxes_stacked]):
    '''
    by default reads the boxes 
    '''
    img_set = []
    for i in range(num_imgs):
        img = Image.open(read_dirs[i] + 'stacked.png')
        img = np.asarray(img)
        img_set.append(img)
    img_set = np.asarray(img_set)
    dataset = tf.data.Dataset.from_tensor_slices(img_set)
    return dataset

In [6]:
data_path = '../../datasets'
disparity = hci_io.read_disparity(data_path + hci_boxes)
print('disparity shape: \n', disparity.shape)
depth = hci_io.read_depth(data_path + hci_boxes)
print('depth shape: \n', depth.shape)

dataset = load_dataset()
for elem in dataset:
    print(elem.numpy().shape)


disparity file read success:  ../../datasets/hci_dataset/training/boxes/gt_disp_lowres.pfm
disparity shape: 
 (512, 512)
depth file read success:  ../../datasets/hci_dataset/training/boxes/gt_depth_lowres.pfm
depth shape: 
 (512, 512)
(3038, 3038, 3)


## Model init

In [7]:

model = keras.Sequential([
keras.layers.Conv2D(input_shape=(28,28,1), filters=8, kernel_size=3, 
                      strides=2, activation='relu', name='Conv1'),
                    keras.layers.Flatten(),
                    keras.layers.Dense(10, name='Dense')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv1 (Conv2D)              (None, 13, 13, 8)         80        
                                                                 
 flatten_1 (Flatten)         (None, 1352)              0         
                                                                 
 Dense (Dense)               (None, 10)                13530     
                                                                 
Total params: 13,610
Trainable params: 13,610
Non-trainable params: 0
_________________________________________________________________
